Try to keep the code here. And then export into a .py file.

Do not try to modify the .py file directly until this is notebook is gone from each branch.

Remember to clear output

In [ ]:
# needed for model

import torch
import torch.nn as nn


In [ ]:
# needed for formulating final predictions

from torchvision.ops import nms


In [ ]:
# yolo v3

# Utils and calculating loss

Load presaved model

In [ ]:
# works by side effects
def load_pretask_weight_from_model(model, presaved_encoder):
    model.encoder.load_state_dict(presaved_encoder.state_dict())
    
    for param in model.encoder.parameters():
        param.requires_grad = False
        
    return model

In [ ]:
# use this if you want Initialize Our Model with encoder weights from an existing pretask encoder in memory
def initialize_model_for_training(presaved_encoder):
    model = KobeModel()
    load_pretask_weight_from_model(model, presaved_encoder)
    
    return model

In [ ]:
# use this if you want Initialize Our Model with encoder weights from a file
def initialize_model_for_training_file(presaved_encoder_file):
    presaved_encoder = PreTaskEncoder()
    presaved_encoder.load_state_dict(torch.load(presaved_encoder_file))
    presaved_encoder.eval()

    
    return initialize_model_for_training(presaved_encoder)

Converting predictions to the format for competition

# Model Part

Our YoloLayer for task of object localization

In [ ]:
class PreTaskEncoder(nn.Module):
    def __init__(self):
        super(PreTaskEncoder, self).__init__()
    

In [ ]:
class YoloLayer(nn.Module):
    
    def __init__(self, num_classes):
        super(PreTaskEncoder, self).__init__()
        
        
        self.m = nn.Sequential(
        
        )
        
    def forward(self, x):
        

Our Model does that does both tasks

In [ ]:
class KobeModel(nn.Module):
    
    def __init__(self):
        super(KobeModel, self).__init__()
        
        # 
        
        self.encoder = PreTaskEncoder()
        
        
        self.shared_decoder = nn.Sequential(
            
        )
        
        self.yolo = YoloLayer()
        
        self.rm_decoder = RmDecoder()
        
    def forward(self, x):
        x = self.encoder(x)
        
        #convert from dense representation from encoder into an image
        x.view(...)
        
        x = self.shared_decoder(x)
        
        # output_1[0] corresponds to the bounding boxes
        # ouptut_1[1] corresponds to the class labels
        output_1 = self.yolo(x)
        # roadmap decoder
        output_2 = self.rm_decoder(x)
        
        return output_1, output_2